<a href="https://colab.research.google.com/github/AndrewZinc/Big_Data/blob/main/Tokenize_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
print('hello1')
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
print('hello2')
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
print('hello3')
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()
print('hello4')

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,554 kB]
Hit:13 http://ppa.launchpa

In [2]:
# start Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('DataFrameFunctions').getOrCreate()

In [3]:
# Import the tokenizer library

from pyspark.ml.feature import Tokenizer


In [4]:
# Create a sample dataframe

df = spark.createDataFrame([
    (0, 'Spark is great'),
    (1, 'We are learning about Spark'),
    (2, 'Spark is no doubt better than Hadoop')
    ], ['id', 'sentence'])

df.show()


+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|      Spark is great|
|  1|We are learning a...|
|  2|Spark is no doubt...|
+---+--------------------+



In [11]:
# tokenize the sentences

tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

tokenizer

Tokenizer_c037f3d1efe0

In [12]:
# Transform and show the DataFrame

tokenized_df = tokenizer.transform(df)
tokenized_df.show(truncate=False)


+---+------------------------------------+--------------------------------------------+
|id |sentence                            |words                                       |
+---+------------------------------------+--------------------------------------------+
|0  |Spark is great                      |[spark, is, great]                          |
|1  |We are learning about Spark         |[we, are, learning, about, spark]           |
|2  |Spark is no doubt better than Hadoop|[spark, is, no, doubt, better, than, hadoop]|
+---+------------------------------------+--------------------------------------------+



In [13]:
# Create a function to return the length of a list
def word_list_length(word_list):
  return len(word_list)
  


In [14]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

# Create a user-defined-function

count_tokens = udf(word_list_length, IntegerType())


In [16]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
tokenized_df = tokenizer.transform(df)
tokenized_df.withColumn('tokens', count_tokens(col('words'))).show(truncate=False)

+---+------------------------------------+--------------------------------------------+------+
|id |sentence                            |words                                       |tokens|
+---+------------------------------------+--------------------------------------------+------+
|0  |Spark is great                      |[spark, is, great]                          |3     |
|1  |We are learning about Spark         |[we, are, learning, about, spark]           |5     |
|2  |Spark is no doubt better than Hadoop|[spark, is, no, doubt, better, than, hadoop]|7     |
+---+------------------------------------+--------------------------------------------+------+

